# Excluding subjects having issues.


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

# load datasets
df_qs = pd.read_csv("ascii-data-files_nida-ctn-0007/qs.csv")
df_qs.drop_duplicates()

df_sc = pd.read_csv("ascii-data-files_nida-ctn-0007/sc.csv")
df_suppds = pd.read_csv("ascii-data-files_nida-ctn-0007/suppds.csv")

# store subject keys to be excluded
excluded_usubjid_list = []
excluded_usubjid_list += list(np.unique(df_sc[(df_sc['SCTEST']=='MET ALL INCLUSION & NO EXCLUSION CRIT.')
                                          & (df_sc['SCORRES']=='NO')]['USUBJID']))
exclusion_reason = ["On --/--/02, RA discovered that patient had previously been enrolled and run in MIEDAR study (before RA's arrival). Patient and counselor were notified of the termination on --/--/02",
                    'Due to lack of documentation, it is unknown whether patient continued participation in the research protocol beyond this date.',
                    'Due to lack of source documentation, patient did not complete protocol and there is no documentation indicating patient continued with study assessments beyond this date.',
                    'There are no source docs or Miedar program entries after wk10/v1. It is unknown if subject attended any study visits after this point. It cannot be determined if subject completed study according to p',
                    'There are no source documents or Miedar program entries after wk8/v2. It is unknown if suject attended any study visits after this point. It cannot be determined if subject completed study according t']
excluded_usubjid_list += list(np.unique(df_suppds[(df_suppds['QNAM']=='DSOTHSP')
                                                  & (df_suppds['QVAL'].isin(exclusion_reason))]['USUBJID']))
excluded_usubjid_list = list(set(excluded_usubjid_list))

for usubjid in np.unique(df_qs['USUBJID']):
    num_of_visit = len(np.unique(df_qs[df_qs['USUBJID']==usubjid]['VISIT']))
    if num_of_visit <= 10:
        excluded_usubjid_list.append(usubjid)
del(usubjid)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# get subjectkeys from control (usual care) and incentive (usual care + contingency management) groups
control_usubjid_list = list(np.unique(df_qs[(df_qs['QSTEST']=='SUBJECT RESEARCH GROUP')
                                            & (df_qs['QSORRES']=='CONTROL')]['USUBJID']))
incentive_usubjid_list = list(np.unique(df_qs[(df_qs['QSTEST']=='SUBJECT RESEARCH GROUP')
                                              & (df_qs['QSORRES']=='INCENTIVE')]['USUBJID']))
control_usubjid_list = list(set(control_usubjid_list)-set(excluded_usubjid_list))
incentive_usubjid_list = list(set(incentive_usubjid_list)-set(excluded_usubjid_list))

for usubjid in set(control_usubjid_list).intersection(set(incentive_usubjid_list)):
    current_subject_status_list = list(df_qs[(df_qs['USUBJID']==usubjid)
                                             & (df_qs['QSSCAT']=='SUBJECT STATUS')]['QSORRES'])
    current_subject_status = max(set(current_subject_status_list),
                                 key=current_subject_status_list.count)
    if current_subject_status == 'CONTROL':
        incentive_usubjid_list.remove(usubjid)
    elif current_subject_status == 'INCENTIVE':
        control_usubjid_list.remove(usubjid)
del(usubjid)
total_usubjid_list = incentive_usubjid_list + control_usubjid_list

print('Sample size for the incentive group: ', len(incentive_usubjid_list))
print('Sample size for the control group: ', len(control_usubjid_list))
# total 388 subjects
df = df_qs[df_qs['USUBJID'].isin(total_usubjid_list)]

Sample size for the incentive group:  195
Sample size for the control group:  188


# Process the dataset

In [3]:
# load the dataset
df_dm = pd.read_csv("ascii-data-files_nida-ctn-0007/dm.csv")

# declare default lists
lottery_result_list = ['NUMBER OF DRAWS', "NUMBER OF 'GOOD JOBS'", 'NUMBER OF SMALL PRIZES',
                       'NUMBER OF LARGE PRIZES', 'NUMBER OF JUMBO PRIZES']
substance_result_list = ['ALCOHOL', 'AMPHETAMINE', 'COCAINE', 'METHAMPHETAMINE', 'OPIOID']
qsscat_list = ['BREATHALYZER RESULTS', 'INCENTIVES TRACKING', 'SUBJECT STATUS', 'URINE RESULTS']
visit_list = ['WEEK 1: 1ST WEEKLY', 'WEEK 1: 2ND WEEKLY', 'WEEK 2: 1ST WEEKLY', 'WEEK 2: 2ND WEEKLY',
             'WEEK 3: 1ST WEEKLY', 'WEEK 3: 2ND WEEKLY', 'WEEK 4: 1ST WEEKLY', 'WEEK 4: 2ND WEEKLY',
             'WEEK 5: 1ST WEEKLY', 'WEEK 5: 2ND WEEKLY', 'WEEK 6: 1ST WEEKLY', 'WEEK 6: 2ND WEEKLY',
             'WEEK 7: 1ST WEEKLY', 'WEEK 7: 2ND WEEKLY', 'WEEK 8: 1ST WEEKLY', 'WEEK 8: 2ND WEEKLY',
             'WEEK 9: 1ST WEEKLY', 'WEEK 9: 2ND WEEKLY', 'WEEK 10: 1ST WEEKLY', 'WEEK 10: 2ND WEEKLY',
             'WEEK 11: 1ST WEEKLY', 'WEEK 11: 2ND WEEKLY', 'WEEK 12: 1ST WEEKLY', 'WEEK 12: 2ND WEEKLY']

# preprocess states, actions, and rewards. This part includes hard-coded fixes for certain subjects based on their ID and visit.
df = df[df['VISIT'].isin(visit_list)]
df = df[df['QSSCAT'].isin(qsscat_list)]

total_df, error_df = [], []
for usubjid in np.unique(df['USUBJID']):    
    selected_df = df[df['USUBJID'] == usubjid]
    selected_df_dm = df_dm[df_dm['USUBJID'] == usubjid]
    for visit in np.unique(selected_df['VISIT']):
        current_df = {}
        current_df['USUBJID'] = usubjid
        epoch = np.unique(selected_df[selected_df['VISIT']==visit]['EPOCH'])[0]
        current_df['EPOCH'] = epoch
        if usubjid in incentive_usubjid_list:
            current_df['SUBJECT RESEARCH GROUP'] = 'INCENTIVE'
        elif usubjid in control_usubjid_list:
            current_df['SUBJECT RESEARCH GROUP'] = 'CONTROL'

        week = int(visit[5:][:np.char.find(visit[5:], ':')])
        if '1ST' in visit:
            round_of_visit = 1
        elif '2ND' in visit:
            round_of_visit = 2

        current_df['WEEK'] = week
        current_df['ROUND OF VISIT'] = round_of_visit
        current_df['AGE'] = np.array(selected_df_dm['AGE'])[0]
        current_df['SEX'] = np.array(selected_df_dm['SEX'])[0]

        attendance_key = 'ATTENDANCE: URINE RESULT'
        attendance = np.array(selected_df[(selected_df['VISIT']==visit)
                                          & (selected_df['QSTEST']==attendance_key)
                                         & (selected_df['EPOCH']==epoch)]['QSORRES'])
        attendance = attendance[0] if len(attendance) >= 1 else 'MISSING'
        current_df[attendance_key] = attendance

        if attendance == 'PRESENT':
            for lottery_result in lottery_result_list:
                summary = selected_df[(selected_df['VISIT']==visit)
                                      & (selected_df['QSTEST']==lottery_result)
                                     & (selected_df['EPOCH']==epoch)]['QSSTRESN']
                if len(np.unique(summary).astype('int'))>1:
                    if usubjid == '07_070332':
                        if lottery_result == 'NUMBER OF DRAWS':
                            current_df[lottery_result] = 1
                        elif lottery_result == 'NUMBER OF SMALL PRIZES':
                            current_df[lottery_result] = 1
                    elif usubjid == '07_092679':
                        if lottery_result == 'NUMBER OF DRAWS':
                            current_df[lottery_result] = 9
                        elif lottery_result == "NUMBER OF 'GOOD JOBS'":
                            current_df[lottery_result] = 4
                        elif lottery_result == 'NUMBER OF SMALL PRIZES':
                            current_df[lottery_result] = 4
                        elif lottery_result == 'NUMBER OF LARGE PRIZES':
                            current_df[lottery_result] = 1
                    elif usubjid == '07_094322':
                        if lottery_result == 'NUMBER OF DRAWS':
                            current_df[lottery_result] = 3
                        elif lottery_result == "NUMBER OF 'GOOD JOBS'":
                            current_df[lottery_result] = 2
                        elif lottery_result == 'NUMBER OF SMALL PRIZES':
                            current_df[lottery_result] = 1
                    else:
                        display(selected_df[(selected_df['VISIT']==visit)
                                            & (selected_df['QSTEST']==lottery_result)
                                            & (selected_df['EPOCH']==epoch)])
                        
                if len(summary) > 0:
                    current_df[lottery_result] = np.unique(summary).astype('int')[0]
                elif len(summary) == 0:
                    current_df[lottery_result] = 0

            for substance_result in substance_result_list:
                if substance_result != 'ALCOHOL':
                    summary = selected_df[(selected_df['VISIT']==visit)
                                          & (selected_df['QSTEST']==substance_result)
                                         & (selected_df['EPOCH']==epoch)]['QSORRES']
                    if len(np.unique(summary))>1:
                        if usubjid == '07_029171':
                            if visit == 'WEEK 4: 1ST WEEKLY':
                                if substance_result == 'OPIOID':
                                    current_df[substance_result] = 'NEGATIVE'
                        elif usubjid == '07_040637':
                            if visit == 'WEEK 3: 2ND WEEKLY':
                                if substance_result == 'OPIOID':
                                    current_df[substance_result] = 'POSITIVE'
                        elif usubjid == '07_070332':
                            if visit == 'WEEK 6: 2ND WEEKLY':
                                if substance_result == 'COCAINE':
                                    current_df[substance_result] = 'POSITIVE'
                        elif usubjid == '07_073442':
                            if visit == 'WEEK 4: 1ST WEEKLY':
                                if substance_result == 'OPIOID':
                                    current_df[substance_result] = 'NEGATIVE'
                            elif visit == 'WEEK 3: 2ND WEEKLY':
                                if substance_result == 'OPIOID':
                                    current_df[substance_result] = 'NEGATIVE'
                            elif visit == 'WEEK 10: 1ST WEEKLY':
                                if substance_result == 'AMPHETAMINE':
                                    current_df[substance_result] = 'NEGATIVE'
                                elif substance_result == 'COCAINE':
                                    current_df[substance_result] = 'NEGATIVE'
                                elif substance_result == 'METHAMPHETAMINE':
                                    current_df[substance_result] = 'NEGATIVE'
                                elif substance_result == 'OPIOID':
                                    current_df[substance_result] = 'NEGATIVE'
                        elif usubjid == '07_041594':
                            if visit == 'WEEK 6: 2ND WEEKLY':
                                if substance_result == 'COCAINE':
                                    current_df[substance_result] = 'POSITIVE'                            
                        elif usubjid == '07_073639':
                            if visit == 'WEEK 1: 2ND WEEKLY':
                                if substance_result == 'OPIOID':
                                    current_df[substance_result] = 'POSITIVE'
                            elif visit == 'WEEK 2: 2ND WEEKLY':
                                if substance_result == 'OPIOID':
                                    current_df[substance_result] = 'POSITIVE'
                            elif visit == 'WEEK 6: 2ND WEEKLY':
                                if substance_result == 'AMPHETAMINE':
                                    current_df[substance_result] = 'NO URINE LEFT'
                                elif substance_result == 'COCAINE':
                                    current_df[substance_result] = 'POSITIVE'
                        elif usubjid == '07_094322':
                            if visit == 'WEEK 2: 2ND WEEKLY':
                                if substance_result == 'COCAINE':
                                    current_df[substance_result] = 'NEGATIVE'
                                elif substance_result == 'OPIOID':
                                    current_df[substance_result] = 'NEGATIVE'
                            elif visit == 'WEEK 5: 2ND WEEKLY':
                                if substance_result == 'COCAINE':
                                    current_df[substance_result] = 'POSITIVE'
                        elif usubjid == '07_095923':
                            if visit == 'WEEK 5: 1ST WEEKLY':
                                if substance_result == 'AMPHETAMINE':
                                    current_df[substance_result] = 'NO URINE LEFT'
                                elif substance_result == 'COCAINE':
                                    current_df[substance_result] = 'NO URINE LEFT'
                                elif substance_result == 'METHAMPHETAMINE':
                                    current_df[substance_result] = 'NO URINE LEFT'
                                elif substance_result == 'OPIOID':
                                    current_df[substance_result] = 'NO URINE LEFT'
                        else:
                            display(selected_df[(selected_df['VISIT']==visit)
                                                & (selected_df['QSTEST']==substance_result)
                                                & (selected_df['EPOCH']==epoch)])
                    elif len(summary) == 1:
                        current_df[substance_result] = np.unique(summary)[0]
                    elif len(summary) == 0:
                        current_df[substance_result] = 'NO URINE LEFT'
                    
                else:
                    summary = selected_df[(selected_df['VISIT']==visit)
                                          & (selected_df['QSSCAT']=='BREATHALYZER RESULTS')
                                         & (selected_df['EPOCH']==epoch)]['QSSTRESN']
                    if len(np.unique(summary))>1:
                        if usubjid == '07_040637':
                            current_df[substance_result] = 'POSITIVE'
                        elif usubjid == '07_070332':
                            current_df[substance_result] = 'NEGATIVE'
                        elif usubjid == '07_078483':
                            current_df[substance_result] = 'POSITIVE'
                        else:
                            display(selected_df[(selected_df['VISIT']==visit)
                                                & (selected_df['QSSCAT']=='BREATHALYZER RESULTS')
                                                & (selected_df['EPOCH']==epoch)])
                    elif len(np.unique(summary))==1:
                        current_df[substance_result] = 'NEGATIVE' if np.array(summary).astype('float')[0] <= 0.009 else 'POSITIVE'
                    else:
                        current_df[substance_result] = 'NO URINE LEFT'
        else:
            for lottery_result in lottery_result_list:
                current_df[lottery_result] = 0
            for substance_result in substance_result_list:
                current_df[substance_result] = attendance

        current_df = pd.DataFrame(current_df, index=[0])
        total_df.append(current_df)
total_df = pd.concat(total_df).sort_values(by=['USUBJID', 'WEEK', 'ROUND OF VISIT'])

# check whether there are subjects not being processed well
print('len(error_df): ', len(error_df))
if len(error_df) > 0:
    error_df = pd.concat(error_df).sort_values(by=['USUBJID'])

# save the processed data
total_df.to_csv('CTN0007_processed.csv', index=False)

len(error_df):  0


# Filling test results from missing visits as positive

In [4]:
import pandas as pd

# Read in the CSV file into a pandas DataFrame
df = pd.read_csv('CTN0007_processed.csv')

# Drop rows with missing values
df = df.dropna()

# Lists of substance result types
substance_result_list = ['ALCOHOL', 'AMPHETAMINE', 'COCAINE', 'METHAMPHETAMINE', 'OPIOID']
primary_substance_list = ['ALCOHOL', 'AMPHETAMINE', 'COCAINE', 'METHAMPHETAMINE']
stimulant_substance_list = ['AMPHETAMINE', 'COCAINE', 'METHAMPHETAMINE']

# Define negative, excused, and positive conditions for PRIMARY SUBSTANCE
df['PRIMARY SUBSTANCE (NEGATIVE)'] = df[primary_substance_list].eq('NEGATIVE').all(axis=1)
df['PRIMARY SUBSTANCE (EXCUSED)'] = df[primary_substance_list].eq('EXCUSED').any(axis=1)
df['PRIMARY SUBSTANCE (POSITIVE)'] = ~(df['PRIMARY SUBSTANCE (NEGATIVE)'] | df['PRIMARY SUBSTANCE (EXCUSED)'])

# Define the PRIMARY SUBSTANCE status
df['PRIMARY SUBSTANCE'] = np.select(
    [
        df['PRIMARY SUBSTANCE (NEGATIVE)'],
        df['PRIMARY SUBSTANCE (EXCUSED)'],
        df['PRIMARY SUBSTANCE (POSITIVE)']
    ],
    ['NEGATIVE', 'EXCUSED', 'POSITIVE']
)

# Define negative, excused, and positive conditions for SECONDARY SUBSTANCE (OPIOID)
df['SECONDARY SUBSTANCE (NEGATIVE)'] = df['OPIOID'].eq('NEGATIVE')
df['SECONDARY SUBSTANCE (EXCUSED)'] = df['OPIOID'].eq('EXCUSED')
df['SECONDARY SUBSTANCE (POSITIVE)'] = ~(df['SECONDARY SUBSTANCE (NEGATIVE)'] | df['SECONDARY SUBSTANCE (EXCUSED)'])

# Define the SECONDARY SUBSTANCE status
df['SECONDARY SUBSTANCE'] = np.select(
    [
        df['SECONDARY SUBSTANCE (NEGATIVE)'],
        df['SECONDARY SUBSTANCE (EXCUSED)'],
        df['SECONDARY SUBSTANCE (POSITIVE)']
    ],
    ['NEGATIVE', 'EXCUSED', 'POSITIVE']
)

# Define negative, excused, and positive conditions for STIMULANT SUBSTANCE
df['STIMULANT SUBSTANCE (NEGATIVE)'] = df[stimulant_substance_list].eq('NEGATIVE').all(axis=1)
df['STIMULANT SUBSTANCE (EXCUSED)'] = df[stimulant_substance_list].eq('EXCUSED').any(axis=1)
df['STIMULANT SUBSTANCE (POSITIVE)'] = ~(df['STIMULANT SUBSTANCE (NEGATIVE)'] | df['STIMULANT SUBSTANCE (EXCUSED)'])

# Define the STIMULANT SUBSTANCE status
df['STIMULANT SUBSTANCE'] = np.select(
    [
        df['STIMULANT SUBSTANCE (NEGATIVE)'],
        df['STIMULANT SUBSTANCE (EXCUSED)'],
        df['STIMULANT SUBSTANCE (POSITIVE)']
    ],
    ['NEGATIVE', 'EXCUSED', 'POSITIVE']
)

# Select and order the columns to be written to the new CSV file
columns_to_keep = ['USUBJID', 'SUBJECT RESEARCH GROUP', 'WEEK', 'ROUND OF VISIT',
                   'PRIMARY SUBSTANCE', 'SECONDARY SUBSTANCE', 'STIMULANT SUBSTANCE',
                   'ALCOHOL', 'NUMBER OF DRAWS', "NUMBER OF 'GOOD JOBS'", 'NUMBER OF SMALL PRIZES',
                   'NUMBER OF LARGE PRIZES', 'NUMBER OF JUMBO PRIZES', 'AGE', 'SEX']
processed_df = df[columns_to_keep]

# Write the processed DataFrame to a new CSV file
processed_df.to_csv('CTN0007_processed.csv', index=False)
